In [36]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [37]:
#df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Transformed.csv")

In [38]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [39]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [40]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [41]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home             1.000000
seasonPointsPerGame_Home     0.301313
fenwickPercentageAvg_Home    0.159233
goalDiffAvg_Home             0.111563
pointsFromGameAvg_Home       0.105887
hitsDiffAvg_Away             0.074182
reboundsForAvg_Home          0.057896
penaltiesAgainstTotal        0.028917
penaltiesForTotal           -0.019003
hitsDiffAvg_Home            -0.039441
reboundsForAvg_Away         -0.059495
pointsFromGameAvg_Away      -0.113037
goalDiffAvg_Away            -0.114056
fenwickPercentageAvg_Away   -0.166292
seasonPointsPerGame_Away    -0.293767
Name: win_or_lose_Home, dtype: float64

In [42]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [43]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [44]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [45]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [46]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [47]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [48]:
# # baseline model
# from tensorflow import keras


# # create model
# base_model = keras.models.Sequential([
#     keras.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#                        ])
    
# # Compile model
# base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [49]:
def build_model(n_hidden=0, n_neurons=35, learning_rate=0.00039447935272202665, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [50]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=0, n_neurons=35, learning_rate=0.00039447935272202665)


In [51]:
# from scipy.stats import reciprocal
# from sklearn.model_selection import RandomizedSearchCV

# param_distribs = {
#     'model__n_hidden': [0,1,2,3],
#     'model__n_neurons': np.arange(1,50),
#     'model__learning_rate' : reciprocal(1e-5, 1e-2)
# }

# rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3, error_score='raise')

# early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
# rnd_search_cv.fit(X_train_prepared,Y_train, epochs=100,
#                  validation_data = (X_valid_prepared, Y_valid),
#                  callbacks=[early_stopping_cb])

In [52]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [53]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [54]:
#model = rnd_search_cv.best_estimator_.model

In [55]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, epochs=100, batch_size=5, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])



KerasClassifier(
	model=<function build_model at 0x00000263A6034EA0>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=binary_crossentropy
	metrics=accuracy
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [56]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [57]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

array([[ 949,  566],
       [ 482, 1247]], dtype=int64)

In [58]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


When claiming is a win, correct % is  0.6878102592388307
Detected % of wins 0.7212261422787739
F1 score is  0.7041219649915302
Training Accuracy score is  0.6769420468557337


In [59]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.6831072749691739
0.6785714285714286


In [60]:
#df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [61]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

ValueError: Found array with 0 sample(s) (shape=(0, 14)) while a minimum of 1 is required.